In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
from transformers import ViTModel, ViTConfig
import numpy as np
from sklearn.metrics import classification_report
import cv2

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # Define the architecture here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.final_conv = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, x):
        # Implement the forward pass here
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.final_conv(x)
        return x

segmentation_model = UNet()
segmentation_model.load_state_dict(torch.load('model (1).pt', map_location=torch.device('cpu')))
segmentation_model.eval()

# Load the Vision Transformer classification model
model_checkpoint = 'google/vit-base-patch16-224-in21k'

class VIT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=2, model_checkpoint='google/vit-base-patch16-224-in21k'):
        super(VIT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.pooler = nn.Linear(config.hidden_size, config.hidden_size)
        self.pooler_activation = nn.Tanh()

    def forward(self, x):
        x = self.vit(x)['last_hidden_state']
        x = self.pooler_activation(self.pooler(x[:, 0, :]))
        output = self.classifier(x)
        return output

classification_model = VIT()
classification_model.load_state_dict(torch.load('weed_detection_model.pth', map_location=torch.device('cpu')))
classification_model.eval()

class_names = ["non-weed", "weed-images"]

# Define the transformations to apply to the input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    
])

# Load and preprocess the image
image_path =  'C:\\Users\\selman\\Downloads\\healthy_07\\healthy_02\\fbbb6a37-333a-4a09-a048-d994f1cb5eb0.JPG'
image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0)

# Perform classification using the Vision Transformer model
# Perform classification using the Vision Transformer model
# Perform classification using the Vision Transformer model
with torch.no_grad():
    classification_output = classification_model(image_tensor)
_, predicted_classes = torch.topk(classification_output, k=2, dim=1)
confidences = torch.softmax(classification_output, dim=1)[0, predicted_classes]

# Extract the top predicted class and its confidence
top_predicted_class = predicted_classes[0, 0].item()
top_predicted_class_name = class_names[top_predicted_class]
top_confidence = confidences[0, 0].item()

# Check if both weed and non-weed classes are present
if 0 in predicted_classes and 1 in predicted_classes:
    second_predicted_class = predicted_classes[0, 1].item()
    second_predicted_class_name = class_names[second_predicted_class]
    second_confidence = confidences[0, 1].item()
else:
    second_predicted_class = None
    second_predicted_class_name = None
    second_confidence = None

# Print the predicted classes and confidences
print("Top Predicted Class:", top_predicted_class_name)
print("Top Confidence:", top_confidence)
if second_predicted_class is not None:
    print("Second Predicted Class:", second_predicted_class_name)
    print("Second Confidence:", second_confidence)

# Perform segmentation using the U-Net model
with torch.no_grad():
    segmentation_output = segmentation_model(image_tensor)

# Process the segmentation output
# Convert the segmentation output to a binary mask
binary_mask = (segmentation_output > 0.5).float()
binary_mask = binary_mask.argmax(dim=1).squeeze().cpu().numpy()

# Create a mask with the blue color for segmented parts
blue_color = np.array([0, 0, 255], dtype=np.uint8)

# Overlay the segmentation mask on the original image
segmented_image = image_tensor.squeeze().permute(1, 2, 0)
segmented_image = segmented_image.cpu().numpy()
segmented_image[binary_mask == 1] = blue_color

# Convert the segmented image back to PIL format
segmented_image = Image.fromarray(segmented_image.astype(np.uint8))


# Display the results
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(image)
ax1.axis('off')
ax1.set_title('Input Image')

ax2.imshow(segmented_image)
ax2.axis('off')
ax2.set_title('Segmented Image')

plt.show()

# Print the predicted classes and confidences
print("Predicted Classes:", predicted_classes)
print("Confidences:", confidences)
# Generate classification report
target_names = class_names
classification_rep = classification_report([top_predicted_class], [top_predicted_class], labels=[0, 1], target_names=target_names,zero_division=0)

# Print classification report
print("Classification Report:")
print(classification_rep)

# Create a list of colors corresponding to each predicted class
colors = ['red', 'green', 'blue', 'yellow', 'orange']  # Add more colors if needed

# Plot the 2D histogram with different colors
plt.hist2d(predicted_classes.squeeze().tolist(), confidences.squeeze().tolist(), bins=[len(class_names), 10], cmap='viridis')

# Set labels and title
plt.xlabel('Predicted Classes')
plt.ylabel('Confidence')
plt.title('Predicted Classes vs. Confidences')

# Add a colorbar
plt.colorbar()

# Add a legend for the colors
legend_labels = class_names
plt.legend(legend_labels)

# Show the plot
plt.show()